In [1]:
import neuprint_functions
import numpy as np
import json
%load_ext autoreload
%autoreload 2 -p

In [2]:
mbon_id = 612371421 #MBON01_R
types, bodyids = neuprint_functions.get_all_neurons_presynaptic_to(mbon_id, supertype='KC')

In [3]:
synapses = neuprint_functions.pull_mb_synapses(bodyids, post_id=mbon_id, only_mb=True, n_iter=4)

  0%|          | 0/6235 [00:00<?, ?it/s]

  0%|          | 0/6384 [00:00<?, ?it/s]

  0%|          | 0/7133 [00:00<?, ?it/s]

  0%|          | 0/7101 [00:00<?, ?it/s]

In [4]:
# Factor to adjust from pixels to um coordinates
coordinates, lengths = neuprint_functions.pre_syn_coord_dict(synapses, bodyids)
print("The Total number of PRE>POST synapses is = "+str(len(synapses))) 
print("The total number of presynaptic KCs is = "+str(len(coordinates))) 
print("The average number of KC>MBON synapses per KC is = "+str(np.mean(lengths)))
coordinates_json = json.dumps(coordinates, indent = 4)
with open(f'synapse_coordinates_scaled_{mbon_id}.json', 'w') as output:
    output.write(coordinates_json)
output.close()

The Total number of PRE>POST synapses is = 26853
The total number of presynaptic KCs is = 984
The average number of KC>MBON synapses per KC is = 27.289634146341463


In [34]:
skel_graph = neuprint_functions.pull_skeleton(mbon_id, skel_format='nx')
skel = neuprint_functions.pull_skeleton(mbon_id, skel_format='pandas')
skel_swc, relabel_dict = neuprint_functions.relabel_skeleton_swc(skel_graph, skel)

Reloading 'neuprint_functions'.


In [ ]:
with open("neuron_skeleton.swc", "w") as output:
    output.write(skel_swc)